In [ ]:
import os
import time
import requests
import openai


openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise RuntimeError("请先设置环境变量 OPENAI_API_KEY")


CLUSTER_DIR    = r"C:\Users\golds\Desktop\Data-Driven Design\clusters"

OUTPUT_ROOT    = r"C:\Users\golds\Desktop\Data-Driven Design\generated4"

N_GENERATIONS  = 10

BATCH_SIZE     = 1
# DALL·E ： "1024x1024" / "1024x1792" / "1792x1024"
IMAGE_SIZE     = "1024x1024"

RETRY_WAIT_SEC = 60

MAX_RETRIES    = 2
# ——————————————————

def is_rate_limit_error(exc: Exception) -> bool:
    msg = str(exc).lower()

    return (
        "rate limit" in msg
        or "429" in msg
    )

def main():
    os.makedirs(OUTPUT_ROOT, exist_ok=True)

    for cluster_name in sorted(os.listdir(CLUSTER_DIR)):
        cluster_path = os.path.join(CLUSTER_DIR, cluster_name)
        if not os.path.isdir(cluster_path):
            continue

        prompt = (
            f"Generate a high-resolution jewelry design inspired by cluster “{cluster_name}”. "
            f"Generate a high-resolution 3D jewelry design focusing on the form and silhouette of the piece, with realistic metal texture. "

        )
        print(f"→ Cluster {cluster_name}:\n   Prompt: {prompt}")

        resp = None
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                resp = openai.images.generate(
                    prompt=prompt,
                    n=BATCH_SIZE,
                    size=IMAGE_SIZE
                )
                break
            except Exception as e:
                if is_rate_limit_error(e) and attempt < MAX_RETRIES:
                    print(f"   🚨 Hit rate limit on {cluster_name} (attempt {attempt}), sleeping {RETRY_WAIT_SEC}s…")
                    time.sleep(RETRY_WAIT_SEC)
                    continue
                else:
                    print(f"   ❌ Generation failed for {cluster_name}: {e}")
                    resp = None
                    break

        if not resp:
            continue

        out_dir = os.path.join(OUTPUT_ROOT, cluster_name)
        os.makedirs(out_dir, exist_ok=True)
        for i, imgobj in enumerate(resp.data):
            url = imgobj.url
            try:
                data = requests.get(url, timeout=15).content
                fname = f"{cluster_name}_{i}.png"
                with open(os.path.join(out_dir, fname), "wb") as f:
                    f.write(data)
            except Exception as e:
                print(f"   ⚠ error {cluster_name}#{i}: {e}")

        print(f"   ✔ save {len(resp.data)} images {out_dir}\n")

if __name__ == "__main__":
    main()


→ Cluster cluster_0:
   Prompt: Generate a high-resolution jewelry design inspired by cluster “cluster_0”. Generate a high-resolution 3D jewelry design focusing on the form and silhouette of the piece, with realistic metal texture. 
   ✔ 已保存 1 张图到 C:\Users\golds\Desktop\Data-Driven Design\generated4\cluster_0

→ Cluster cluster_1:
   Prompt: Generate a high-resolution jewelry design inspired by cluster “cluster_1”. Generate a high-resolution 3D jewelry design focusing on the form and silhouette of the piece, with realistic metal texture. 
   ✔ 已保存 1 张图到 C:\Users\golds\Desktop\Data-Driven Design\generated4\cluster_1

→ Cluster cluster_2:
   Prompt: Generate a high-resolution jewelry design inspired by cluster “cluster_2”. Generate a high-resolution 3D jewelry design focusing on the form and silhouette of the piece, with realistic metal texture. 
   ✔ 已保存 1 张图到 C:\Users\golds\Desktop\Data-Driven Design\generated4\cluster_2

→ Cluster cluster_3:
   Prompt: Generate a high-resolution jewel